In [1]:
import urllib
from bs4 import BeautifulSoup
import pandas as pd
from collections import Counter
import glob
import os
import re
import numpy as np
import string
from tqdm import tqdm

# Categorizing Judgments

In [2]:
#Setting up some basic stuff to run Topic Modelling
import logging
import itertools

import numpy as np
import gensim

logging.basicConfig(format='%(levelname)s : %(message)s', level=logging.INFO)
logging.root.level = logging.INFO  # ipython sometimes messes up the logging setup; restore

def head(stream, n=10):
    """Convenience fnc: return the first `n` elements of the stream, as plain list."""
    return list(itertools.islice(stream, n))

c:\users\d110489\appdata\local\programs\python\python37-32\lib\site-packages\gensim\utils.py:1209: UserWarning: detected Windows; aliasing chunkize to chunkize_serial
  warnings.warn("detected Windows; aliasing chunkize to chunkize_serial")


In [3]:
#Setting up some basic stuff to run Topic Modelling
import logging
logging.basicConfig(format='%(asctime)s : %(levelname)s : %(message)s', level=logging.INFO)

In [4]:
import os
import tempfile
TEMP_FOLDER = tempfile.gettempdir()
print('Folder "{}" will be used to save temporary dictionary and corpus.'.format(TEMP_FOLDER))

Folder "C:\Users\D110489\AppData\Local\Temp" will be used to save temporary dictionary and corpus.


In [5]:
from gensim import corpora

In [6]:
from gensim.parsing.preprocessing import STOPWORDS
from gensim.parsing.preprocessing import remove_stopwords,preprocess_documents,strip_punctuation,strip_non_alphanum,strip_numeric
from pprint import pprint

In [7]:
legalstopwords=['court','case','order','appeal','petitioner','judge','appellant','section','case','dated','respondent','petitioner','high','act','the','in','a','it','no']
#Code to tokenize judgments into words. We only take words that occur more than five times. 
def tokenize(document):
    document=strip_punctuation(document)
    document=remove_stopwords(document)
    document=strip_non_alphanum(document)
    document=strip_numeric(document)
    text= [word for word in document.lower().split() if word not in legalstopwords]
    from collections import defaultdict
    frequency = defaultdict(int)
    for token in text:
        frequency[token] += 1
    text = [token for token in text if frequency[token] > 5 and len(token)>3]
#     text_frequency=dict((k, v) for k, v in frequency.items() if v >= 5)
#     return text_frequency
    return text

In [8]:
#We assemble the corpus necessary for topic modelling
texts=[]
text_order=[]

for file in tqdm(glob.glob('C:\\Users\\D110489\\Desktop\\Projects\\Supreme Court\\new\\html_docs\\*.html')):
    category=None
    with open(file,'r', encoding='utf8') as infile:
        html = BeautifulSoup(infile, "html.parser")
                       
        judgement = html.find_all('p')
        data={
            'id':file.split("\\")[-1][:-5]
        }
        output = ' '.join([item.text for item in judgement])

        judgment=output
        text=tokenize(judgment)
        texts.append(text)
        text_order.append(data)
        

100%|████████████████████████████████████| 50672/50672 [10:33<00:00, 80.01it/s]


In [9]:
#We push this into a table
df5=pd.DataFrame(text_order)

In [10]:
dictionary = corpora.Dictionary(texts)
print(dictionary)

INFO : adding document #0 to Dictionary(0 unique tokens: [])
INFO : adding document #10000 to Dictionary(18593 unique tokens: ['accused', 'babu', 'learned', 'office', 'patil']...)
INFO : adding document #20000 to Dictionary(26146 unique tokens: ['accused', 'babu', 'learned', 'office', 'patil']...)
INFO : adding document #30000 to Dictionary(31864 unique tokens: ['accused', 'babu', 'learned', 'office', 'patil']...)
INFO : adding document #40000 to Dictionary(36261 unique tokens: ['accused', 'babu', 'learned', 'office', 'patil']...)
INFO : adding document #50000 to Dictionary(39874 unique tokens: ['accused', 'babu', 'learned', 'office', 'patil']...)
INFO : built Dictionary(40075 unique tokens: ['accused', 'babu', 'learned', 'office', 'patil']...) from 50672 documents (total 27024685 corpus positions)


Dictionary(40075 unique tokens: ['accused', 'babu', 'learned', 'office', 'patil']...)


In [11]:
# We assemble the corpus using the inbuilt doc2bow function and store the matrix to a temporary place
corpus = [dictionary.doc2bow(text) for text in texts]
corpora.MmCorpus.serialize(os.path.join(TEMP_FOLDER, 'judgments.mm'), corpus) 

INFO : storing corpus in Matrix Market format to C:\Users\D110489\AppData\Local\Temp\judgments.mm
INFO : saving sparse matrix to C:\Users\D110489\AppData\Local\Temp\judgments.mm
INFO : PROGRESS: saving document #0
INFO : PROGRESS: saving document #1000
INFO : PROGRESS: saving document #2000
INFO : PROGRESS: saving document #3000
INFO : PROGRESS: saving document #4000
INFO : PROGRESS: saving document #5000
INFO : PROGRESS: saving document #6000
INFO : PROGRESS: saving document #7000
INFO : PROGRESS: saving document #8000
INFO : PROGRESS: saving document #9000
INFO : PROGRESS: saving document #10000
INFO : PROGRESS: saving document #11000
INFO : PROGRESS: saving document #12000
INFO : PROGRESS: saving document #13000
INFO : PROGRESS: saving document #14000
INFO : PROGRESS: saving document #15000
INFO : PROGRESS: saving document #16000
INFO : PROGRESS: saving document #17000
INFO : PROGRESS: saving document #18000
INFO : PROGRESS: saving document #19000
INFO : PROGRESS: saving document #2

In [12]:
#We run the LDA model. We can define the number of topics and how often we want to run passes
from gensim.models import LdaModel
ldamodel = LdaModel(corpus=corpus, num_topics=10, id2word=dictionary,passes=20)

INFO : using symmetric alpha at 0.1
INFO : using symmetric eta at 0.1
INFO : using serial LDA version on this node
INFO : running online (multi-pass) LDA training, 10 topics, 20 passes over the supplied corpus of 50672 documents, updating model once every 2000 documents, evaluating perplexity every 20000 documents, iterating 50x with a convergence threshold of 0.001000
INFO : PROGRESS: pass 0, at document #2000/50672
INFO : merging changes from 2000 documents into a model of 50672 documents
INFO : topic #8 (0.100): 0.022*"state" + 0.011*"government" + 0.010*"said" + 0.009*"suit" + 0.007*"held" + 0.007*"property" + 0.007*"public" + 0.007*"judgment" + 0.006*"learned" + 0.006*"filed"
INFO : topic #4 (0.100): 0.014*"state" + 0.009*"said" + 0.009*"government" + 0.008*"land" + 0.008*"india" + 0.008*"time" + 0.007*"evidence" + 0.006*"learned" + 0.006*"income" + 0.006*"held"
INFO : topic #0 (0.100): 0.015*"state" + 0.014*"government" + 0.011*"land" + 0.009*"said" + 0.007*"filed" + 0.006*"held"

INFO : topic #3 (0.100): 0.031*"income" + 0.026*"company" + 0.022*"goods" + 0.022*"assessee" + 0.017*"tribunal" + 0.013*"business" + 0.012*"assessment" + 0.012*"said" + 0.011*"question" + 0.009*"year"
INFO : topic #6 (0.100): 0.047*"accused" + 0.041*"singh" + 0.033*"evidence" + 0.025*"deceased" + 0.016*"prosecution" + 0.012*"persons" + 0.012*"death" + 0.012*"police" + 0.012*"appellants" + 0.011*"witnesses"
INFO : topic diff=0.510166, rho=0.377964
INFO : PROGRESS: pass 0, at document #16000/50672
INFO : merging changes from 2000 documents into a model of 50672 documents
INFO : topic #3 (0.100): 0.030*"income" + 0.027*"company" + 0.024*"goods" + 0.021*"assessee" + 0.016*"tribunal" + 0.012*"business" + 0.011*"said" + 0.011*"question" + 0.011*"assessment" + 0.010*"sales"
INFO : topic #7 (0.100): 0.030*"state" + 0.014*"government" + 0.013*"rules" + 0.012*"rule" + 0.010*"constitution" + 0.009*"service" + 0.009*"post" + 0.009*"list" + 0.009*"article" + 0.008*"shall"
INFO : topic #5 (0.100): 0

INFO : topic #5 (0.100): 0.039*"suit" + 0.034*"property" + 0.022*"plaintiff" + 0.021*"possession" + 0.020*"decree" + 0.016*"defendant" + 0.014*"said" + 0.013*"sale" + 0.011*"land" + 0.010*"right"
INFO : topic #0 (0.100): 0.029*"land" + 0.028*"government" + 0.023*"state" + 0.020*"election" + 0.011*"petition" + 0.011*"detention" + 0.011*"acquisition" + 0.010*"said" + 0.009*"area" + 0.008*"shall"
INFO : topic #4 (0.100): 0.023*"land" + 0.013*"compensation" + 0.012*"rent" + 0.011*"shall" + 0.010*"building" + 0.010*"state" + 0.010*"value" + 0.010*"tenant" + 0.009*"said" + 0.009*"premises"
INFO : topic diff=0.425658, rho=0.267261
INFO : PROGRESS: pass 0, at document #30000/50672
INFO : merging changes from 2000 documents into a model of 50672 documents
INFO : topic #6 (0.100): 0.050*"accused" + 0.041*"singh" + 0.038*"evidence" + 0.031*"deceased" + 0.017*"prosecution" + 0.014*"appellants" + 0.014*"death" + 0.013*"house" + 0.012*"persons" + 0.012*"witnesses"
INFO : topic #5 (0.100): 0.039*"sui

INFO : topic #9 (0.100): 0.021*"constitution" + 0.020*"article" + 0.019*"state" + 0.015*"power" + 0.013*"india" + 0.010*"government" + 0.009*"right" + 0.007*"provisions" + 0.007*"parliament" + 0.006*"rights"
INFO : topic #5 (0.100): 0.041*"suit" + 0.039*"property" + 0.021*"plaintiff" + 0.020*"possession" + 0.020*"decree" + 0.015*"defendant" + 0.014*"right" + 0.014*"said" + 0.012*"sale" + 0.011*"land"
INFO : topic #3 (0.100): 0.030*"income" + 0.027*"company" + 0.024*"goods" + 0.020*"assessee" + 0.013*"sales" + 0.012*"business" + 0.012*"assessment" + 0.011*"tribunal" + 0.010*"said" + 0.010*"sale"
INFO : topic #1 (0.100): 0.023*"accused" + 0.023*"evidence" + 0.020*"police" + 0.016*"offence" + 0.014*"magistrate" + 0.014*"criminal" + 0.013*"trial" + 0.012*"person" + 0.012*"code" + 0.011*"said"
INFO : topic diff=0.356219, rho=0.218218
INFO : PROGRESS: pass 0, at document #44000/50672
INFO : merging changes from 2000 documents into a model of 50672 documents
INFO : topic #6 (0.100): 0.052*"ac

INFO : topic #5 (0.100): 0.043*"suit" + 0.038*"property" + 0.022*"plaintiff" + 0.021*"possession" + 0.021*"decree" + 0.018*"defendant" + 0.014*"sale" + 0.014*"said" + 0.013*"right" + 0.012*"land"
INFO : topic #2 (0.100): 0.020*"tribunal" + 0.018*"service" + 0.016*"award" + 0.015*"government" + 0.015*"employees" + 0.013*"contract" + 0.013*"industrial" + 0.011*"workmen" + 0.011*"said" + 0.011*"corporation"
INFO : topic #6 (0.100): 0.052*"accused" + 0.046*"singh" + 0.043*"evidence" + 0.033*"deceased" + 0.018*"prosecution" + 0.014*"appellants" + 0.014*"death" + 0.013*"witnesses" + 0.013*"house" + 0.013*"persons"
INFO : topic #7 (0.100): 0.030*"state" + 0.021*"rules" + 0.020*"government" + 0.020*"service" + 0.015*"rule" + 0.014*"post" + 0.011*"list" + 0.011*"appointment" + 0.010*"posts" + 0.010*"promotion"
INFO : topic #4 (0.100): 0.029*"land" + 0.017*"rent" + 0.015*"tenant" + 0.015*"compensation" + 0.012*"state" + 0.012*"shall" + 0.011*"premises" + 0.011*"price" + 0.010*"building" + 0.010*

...

INFO : topic #5 (0.100): 0.049*"suit" + 0.041*"property" + 0.023*"plaintiff" + 0.023*"decree" + 0.018*"defendant" + 0.017*"possession" + 0.014*"right" + 0.013*"sale" + 0.013*"family" + 0.012*"said"
INFO : topic #2 (0.100): 0.020*"contract" + 0.018*"tribunal" + 0.017*"award" + 0.016*"employees" + 0.015*"industrial" + 0.014*"agreement" + 0.013*"service" + 0.012*"corporation" + 0.012*"company" + 0.012*"arbitration"
INFO : topic #0 (0.100): 0.055*"government" + 0.048*"state" + 0.020*"election" + 0.013*"notification" + 0.012*"board" + 0.012*"petition" + 0.012*"public" + 0.012*"detention" + 0.011*"central" + 0.009*"committee"
INFO : topic #4 (0.100): 0.064*"land" + 0.018*"compensation" + 0.017*"rent" + 0.017*"tenant" + 0.015*"acquisition" + 0.012*"premises" + 0.012*"shall" + 0.011*"state" + 0.011*"lands" + 0.011*"building"
INFO : topic diff=0.046470, rho=0.150183
INFO : PROGRESS: pass 18, at document #50000/50672
INFO : merging changes from 2000 documents into a model of 50672 documents
INFO

INFO : merging changes from 2000 documents into a model of 50672 documents
INFO : topic #1 (0.100): 0.024*"accused" + 0.021*"criminal" + 0.021*"police" + 0.021*"offence" + 0.017*"code" + 0.016*"evidence" + 0.016*"magistrate" + 0.014*"person" + 0.012*"trial" + 0.012*"charge"
INFO : topic #3 (0.100): 0.031*"income" + 0.022*"goods" + 0.022*"company" + 0.017*"assessee" + 0.012*"assessment" + 0.012*"business" + 0.012*"duty" + 0.011*"sale" + 0.011*"year" + 0.010*"said"
INFO : topic #5 (0.100): 0.049*"suit" + 0.042*"property" + 0.025*"decree" + 0.023*"plaintiff" + 0.018*"defendant" + 0.017*"possession" + 0.015*"sale" + 0.014*"right" + 0.013*"said" + 0.012*"family"
INFO : topic #6 (0.100): 0.051*"accused" + 0.046*"evidence" + 0.043*"singh" + 0.031*"deceased" + 0.018*"prosecution" + 0.016*"death" + 0.014*"witnesses" + 0.013*"appellants" + 0.013*"house" + 0.013*"trial"
INFO : topic #8 (0.100): 0.025*"filed" + 0.023*"application" + 0.023*"petition" + 0.022*"learned" + 0.018*"said" + 0.017*"judgme

INFO : topic #4 (0.100): 0.065*"land" + 0.019*"rent" + 0.017*"compensation" + 0.017*"tenant" + 0.015*"acquisition" + 0.012*"premises" + 0.012*"state" + 0.012*"shall" + 0.011*"lands" + 0.010*"said"
INFO : topic diff=0.040867, rho=0.148518
INFO : PROGRESS: pass 19, at document #24000/50672
INFO : merging changes from 2000 documents into a model of 50672 documents
INFO : topic #8 (0.100): 0.024*"filed" + 0.023*"petition" + 0.023*"application" + 0.022*"learned" + 0.018*"said" + 0.017*"judgment" + 0.017*"proceedings" + 0.016*"passed" + 0.016*"jurisdiction" + 0.015*"writ"
INFO : topic #9 (0.100): 0.023*"article" + 0.022*"state" + 0.021*"constitution" + 0.014*"power" + 0.013*"india" + 0.011*"right" + 0.010*"provisions" + 0.007*"government" + 0.006*"legislature" + 0.006*"rights"
INFO : topic #6 (0.100): 0.052*"accused" + 0.046*"evidence" + 0.043*"singh" + 0.032*"deceased" + 0.018*"prosecution" + 0.015*"death" + 0.014*"witnesses" + 0.014*"appellants" + 0.014*"house" + 0.012*"trial"
INFO : topic

INFO : topic #7 (0.100): 0.028*"service" + 0.023*"rules" + 0.022*"state" + 0.022*"government" + 0.017*"rule" + 0.017*"post" + 0.013*"appointment" + 0.010*"candidates" + 0.010*"posts" + 0.009*"promotion"
INFO : topic #0 (0.100): 0.054*"government" + 0.048*"state" + 0.021*"election" + 0.012*"detention" + 0.012*"board" + 0.012*"petition" + 0.012*"notification" + 0.011*"public" + 0.011*"central" + 0.009*"shall"
INFO : topic diff=0.044136, rho=0.148518
INFO : PROGRESS: pass 19, at document #38000/50672
INFO : merging changes from 2000 documents into a model of 50672 documents
INFO : topic #0 (0.100): 0.054*"government" + 0.047*"state" + 0.021*"election" + 0.012*"detention" + 0.012*"board" + 0.012*"petition" + 0.012*"notification" + 0.011*"central" + 0.011*"public" + 0.009*"shall"
INFO : topic #4 (0.100): 0.062*"land" + 0.019*"compensation" + 0.017*"rent" + 0.017*"tenant" + 0.015*"acquisition" + 0.013*"premises" + 0.012*"state" + 0.011*"shall" + 0.011*"lands" + 0.010*"said"
INFO : topic #1 (

INFO : topic #6 (0.100): 0.051*"accused" + 0.047*"evidence" + 0.046*"singh" + 0.031*"deceased" + 0.017*"prosecution" + 0.014*"witnesses" + 0.014*"death" + 0.014*"appellants" + 0.013*"trial" + 0.013*"house"
INFO : topic #9 (0.100): 0.023*"article" + 0.023*"constitution" + 0.022*"state" + 0.015*"power" + 0.013*"india" + 0.011*"right" + 0.010*"provisions" + 0.007*"government" + 0.007*"rights" + 0.006*"legislature"
INFO : topic #3 (0.100): 0.031*"income" + 0.023*"goods" + 0.022*"company" + 0.018*"assessee" + 0.013*"assessment" + 0.012*"sale" + 0.012*"duty" + 0.012*"business" + 0.011*"sales" + 0.010*"said"
INFO : topic #2 (0.100): 0.020*"contract" + 0.018*"tribunal" + 0.017*"award" + 0.015*"employees" + 0.015*"industrial" + 0.014*"agreement" + 0.013*"service" + 0.013*"corporation" + 0.012*"company" + 0.012*"arbitration"
INFO : topic diff=0.038190, rho=0.148518
INFO : -6.836 per-word bound, 114.2 perplexity estimate based on a held-out corpus of 672 documents with 330608 words
INFO : PROGRES

In [13]:
ldamodel.print_topics(10)

INFO : topic #0 (0.100): 0.055*"government" + 0.047*"state" + 0.019*"election" + 0.012*"notification" + 0.012*"public" + 0.012*"detention" + 0.012*"board" + 0.012*"petition" + 0.011*"central" + 0.009*"committee"
INFO : topic #1 (0.100): 0.024*"accused" + 0.023*"police" + 0.022*"criminal" + 0.021*"offence" + 0.017*"code" + 0.017*"magistrate" + 0.016*"evidence" + 0.015*"person" + 0.013*"trial" + 0.012*"charge"
INFO : topic #2 (0.100): 0.021*"contract" + 0.018*"tribunal" + 0.018*"award" + 0.016*"industrial" + 0.015*"employees" + 0.014*"agreement" + 0.013*"service" + 0.012*"corporation" + 0.012*"company" + 0.012*"workmen"
INFO : topic #3 (0.100): 0.031*"income" + 0.023*"goods" + 0.021*"company" + 0.018*"assessee" + 0.012*"business" + 0.012*"assessment" + 0.012*"sale" + 0.012*"duty" + 0.011*"said" + 0.011*"sales"
INFO : topic #4 (0.100): 0.065*"land" + 0.018*"rent" + 0.018*"tenant" + 0.017*"compensation" + 0.014*"acquisition" + 0.012*"premises" + 0.012*"shall" + 0.012*"state" + 0.011*"lands

[(0,
  '0.055*"government" + 0.047*"state" + 0.019*"election" + 0.012*"notification" + 0.012*"public" + 0.012*"detention" + 0.012*"board" + 0.012*"petition" + 0.011*"central" + 0.009*"committee"'),
 (1,
  '0.024*"accused" + 0.023*"police" + 0.022*"criminal" + 0.021*"offence" + 0.017*"code" + 0.017*"magistrate" + 0.016*"evidence" + 0.015*"person" + 0.013*"trial" + 0.012*"charge"'),
 (2,
  '0.021*"contract" + 0.018*"tribunal" + 0.018*"award" + 0.016*"industrial" + 0.015*"employees" + 0.014*"agreement" + 0.013*"service" + 0.012*"corporation" + 0.012*"company" + 0.012*"workmen"'),
 (3,
  '0.031*"income" + 0.023*"goods" + 0.021*"company" + 0.018*"assessee" + 0.012*"business" + 0.012*"assessment" + 0.012*"sale" + 0.012*"duty" + 0.011*"said" + 0.011*"sales"'),
 (4,
  '0.065*"land" + 0.018*"rent" + 0.018*"tenant" + 0.017*"compensation" + 0.014*"acquisition" + 0.012*"premises" + 0.012*"shall" + 0.012*"state" + 0.011*"lands" + 0.010*"said"'),
 (5,
  '0.048*"suit" + 0.041*"property" + 0.023*"plai

In [14]:
lda_topic_assignment = [max(p,key=lambda item: item[1]) for p in ldamodel[corpus]]


In [15]:
lda_topic_assignment

[(1, 0.5729598),
 (4, 0.45959032),
 (6, 0.9973447),
 (7, 0.58070624),
 (5, 0.66904765),
 (6, 0.8714236),
 (3, 0.7226098),
 (4, 0.68689406),
 (3, 0.58522826),
 (4, 0.9790695),
 (1, 0.61472756),
 (4, 0.78826773),
 (8, 0.9437428),
 (9, 0.44388536),
 (4, 0.59941924),
 (7, 0.86450917),
 (8, 0.43689558),
 (4, 0.39525482),
 (1, 0.5674976),
 (7, 0.32242623),
 (5, 0.8313253),
 (0, 0.1),
 (7, 0.68325245),
 (6, 0.8105227),
 (4, 0.74762),
 (4, 0.44056582),
 (7, 0.9404075),
 (3, 0.9247671),
 (3, 0.78317446),
 (7, 0.9886082),
 (2, 0.52582973),
 (3, 0.30795965),
 (0, 0.86207134),
 (0, 0.1),
 (4, 0.5177618),
 (3, 0.5515893),
 (2, 0.49873245),
 (7, 0.8850717),
 (2, 0.9412284),
 (3, 0.6771666),
 (1, 0.7521625),
 (7, 0.90679157),
 (7, 0.47828078),
 (5, 0.5578188),
 (4, 0.7022992),
 (6, 0.9819999),
 (8, 0.9016455),
 (3, 0.5656981),
 (2, 0.98830926),
 (1, 0.5686414),
 (0, 0.1),
 (8, 0.57024056),
 (1, 0.6970595),
 (5, 0.581983),
 (0, 0.704105),
 (5, 0.655371),
 (4, 0.5642094),
 (1, 0.56069845),
 (2, 0.94705

In [16]:
df5['category']=lda_topic_assignment

In [17]:
df5.to_csv("classifieddata3.csv")